In [ ]:
import polars as pl
from math import sqrt, atan2, pi, acos
import random
import rtsvg
rt = rtsvg.RACETrack()
from circle_packer import CirclePacker
import os
import kagglehub
path = kagglehub.dataset_download("konivat/tree-of-life")
dfs = []
for _filename_ in os.listdir(path):
    df = pl.read_csv(os.path.join(path, _filename_))
    dfs.append(df)

_circles_ = [(0.0, 0.0, 1.3225698144771698),
 (0.0, 0.0, 22.609033029142832),
 (0.0, 0.0, 14.300643684166218),
 (0.0, 0.0, 27.50857310783749),
 (0.0, 0.0, 13.502128545161003),
 (0.0, 0.0, 6.751248480523149),
 (0.0, 0.0, 18.193088691788525),
 (0.0, 0.0, 28.613640274927544),
 (0.0, 0.0, 19.251574555620248),
 (0.0, 0.0, 26.66200202886573),
 (0.0, 0.0, 28.974697297418327),
 (0.0, 0.0, 27.08516744671632),
 (0.0, 0.0, 4.94923810208677),
 (0.0, 0.0, 4.212546335061312),
 (0.0, 0.0, 4.823746665552136),
 (0.0, 0.0, 21.008164042193926),
 (0.0, 0.0, 17.54002285560996),
 (0.0, 0.0, 22.131382136912297),
 (0.0, 0.0, 4.321217609157426),
 (0.0, 0.0, 15.123595665202778)]
_cp_ = CirclePacker(rt, _circles_[:5])
_cp_.pack()
rt.table(_cp_.progression, per_row=4, spacer=10)    

In [ ]:
#
# 28.0s for 100_000 circles on M1 Pro (Unoptimized)
#  2.9s for 100_000 circles on M1 Pro (w/ Arc Length Optimization)
#
_tiles_ = []
_list_  = []
for i in range(1):
    _circles_ = []
    for i in range(1_000): _circles_.append((0.0, 0.0, 0.1 + 10.0*random.random()))
    _list_.append(_circles_)
    _cp_ = CirclePacker(rt, _circles_, capture_progression=False, validate_chains=False)
    _cp_.pack()
    _tiles_.append(_cp_)
rt.table(_tiles_, spacer=10, per_row=6)

In [ ]:
_false_examples_     = []
_befores_and_afters_ = []
_tiles_ = []
_lu_    = {'mult':[], 'validated':[], 'sample':[], 'contains_before_and_afters':[]}
for _multiplier_ in range(0, 20, 4):
    print('.',end='')
    for _sample_ in range(1_000):
        _circles_ = []
        for i in range(20): _circles_.append((0.0, 0.0, 1.0 + 2.0*_multiplier_*random.random()))
        _list_.append(_circles_)
        _cp_ = CirclePacker(rt, _circles_, check_overlaps_every_append=True)
        _cp_.pack()
        _tiles_.append(_cp_)
        if _cp_.__validateNoOverlaps__():  
            _lu_['validated'].append(True)
        else:
            print('!',end='')
            _lu_['validated'].append(False)
            _false_examples_.append(_cp_)
            _befores_and_afters_.extend(_cp_.before_and_afters)
        _lu_['contains_before_and_afters'].append(len(_cp_.before_and_afters) > 0)
        _lu_['mult'].append(_multiplier_)
        _lu_['sample'].append(_sample_)
_df_          = pl.DataFrame(_lu_)
_df_true_     = _df_.filter(pl.col('validated') == True)
_df_false_    = _df_.filter(pl.col('validated') == False)
_histo_tiles_ = [rt.histogram(_df_, bin_by='mult', color_by='validated')]

if len(_df_true_)  > 0: _histo_tiles_.append(rt.histogram(_df_true_,  bin_by='mult', color_by='contains_before_and_afters'))
if len(_df_false_) > 0: _histo_tiles_.append(rt.histogram(_df_false_, bin_by='mult', color_by='contains_before_and_afters'))
rt.tile(_histo_tiles_)

In [ ]:
rt.table(_false_examples_)

In [ ]:
_counter_ = {}
for _cp_ in _false_examples_: 
    x = _cp_.overlap_found_at
    if x not in _counter_: _counter_[x] = 0
    _counter_[x] += 1
print(_counter_)

In [ ]:
_false_examples_[-1].circles